In [1]:
import sys, os

src = r"C:\Users\user\Desktop\Coding mo\AutoML proj1\task 1\AutoML Project Molham\src" # copy the location of the folder that has paths.py file using (ctrl + shift + C ) and paste it here    
sys.path.append(src)

from paths import *

get_paths()

sys.path.append(SRC_PATH)

from pipeline import *





In [10]:
# ---------------- Task 1+9 ----------------

import h2o
from h2o.automl import H2OAutoML
import torch

os.environ["JAVA_HOME"] = r"C:\Users\user\anaconda3\envs\automl_h2o"
os.environ["PATH"] = os.path.join(os.environ["JAVA_HOME"], "Library", "bin") + ";" + os.environ["PATH"]

def loop_dfs_and_evaluate_h2o():
    h2o.init(max_mem_size="4G", nthreads=-1)
    h2o.remove_all()

    for dataset_file in data_dict_classification_only.keys():
        dataset_summary = []
        runtime_memory_rows = []
        dataset_base = os.path.splitext(dataset_file)[0]

        for seed in RANDOM_SEEDS:
            np.random.seed(seed)
            random.seed(seed)
            try:
                torch.manual_seed(seed)
                torch.cuda.empty_cache()
            except:
                pass

            df_train = splits_dict[dataset_file]['train']
            df_test  = splits_dict[dataset_file]['test']

            h2o_train = h2o.H2OFrame(df_train)
            h2o_train[TARGET_COLS[dataset_file]] = h2o_train[TARGET_COLS[dataset_file]].asfactor()

            start_time = time.time()
            aml = H2OAutoML(max_runtime_secs=TIME_BUDGET, seed=seed, project_name=f"{dataset_base}_{seed}_{int(time.time())}")
            aml.train(y=TARGET_COLS[dataset_file], training_frame=h2o_train)
            training_runtime_sec = round(time.time() - start_time, 3)

            infer_start = time.time()
            results = evaluate_and_save_results_general(df_train, df_test, aml, dataset_file, framework='h2o')
            inference_time_per_sample = round((time.time() - infer_start) / len(df_test), 6)
            cpu_usage_percent = psutil.cpu_percent(interval=None)

            temp_model_dir = os.path.join("temp_h2o_models", f"{dataset_base}_{seed}")
            os.makedirs(temp_model_dir, exist_ok=True)
            h2o.save_model(model=aml.leader, path=temp_model_dir, force=True)

            total_bytes = 0
            for root, dirs, files in os.walk(temp_model_dir):
                for f in files:
                    total_bytes += os.path.getsize(os.path.join(root, f))
            model_size_mb = round(total_bytes / (1024 * 1024), 3)
            shutil.rmtree(temp_model_dir)

            memory_usage_mb = get_memory_usage_mb()

            runtime_memory_rows.append({
                "dataset": dataset_file,
                "seed": seed,
                "training_runtime_sec": training_runtime_sec,
                "model_size_MB": model_size_mb,
                "memory_usage_MB": memory_usage_mb,
                "inference_time_per_sample_sec": inference_time_per_sample,
                "cpu_usage_percent": cpu_usage_percent
            })

            dataset_summary.append({
                'dataset': dataset_file,
                'seed': seed,
                **results,
                'runtime': training_runtime_sec
            })

            del aml, h2o_train
            gc.collect()

        out_path = os.path.join(AUTOML_METRICS_PATH, f"h2o_{dataset_base}_summary.csv")
        pd.DataFrame(dataset_summary).to_csv(out_path, index=False)

        task1_output_path = os.path.join(RUNTIME_MEMORY_TASK1_PATH, f"h2o_{dataset_base}_task1_runtime_memory_metrics.csv")
        pd.DataFrame(runtime_memory_rows).to_csv(task1_output_path, index=False)

        json_path = os.path.join(RUNTIME_MEMORY_TASK1_PATH, f"h2o_{dataset_base}_runtime_memory_metrics_in_json.json")
        with open(json_path, 'w') as f:
            json.dump(runtime_memory_rows, f, indent=2)

    h2o.shutdown(prompt=False)

loop_dfs_and_evaluate_h2o()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Zulu25.30+17-CA (build 25.0.1+8-LTS, mixed mode, sharing)
  Starting server from C:\Users\user\anaconda3\envs\automl_h2o\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\user\AppData\Local\Temp\tmp_3lym2wz
  JVM stdout: C:\Users\user\AppData\Local\Temp\tmp_3lym2wz\h2o_user_started_from_python.out
  JVM stderr: C:\Users\user\AppData\Local\Temp\tmp_3lym2wz\h2o_user_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Amman
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.8
H2O_cluster_version_age:,1 month and 26 days
H2O_cluster_name:,H2O_from_python_user_3h5qh6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.983 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
٠٤:٢٧:٢٠.٣٩٩: AutoML: XGBoost is not available; skipping it.
٠٤:٢٧:٢٠.٤٣٠: _train param, Dropping bad and constant columns: [AHG, Stillwater_Insurance_Group, BC, Selective_Ins_Co_of_America_Selective_, C_N_A_Surety, Commerce_West_Insurance_Company, Plaza_Insurance_Company_State_Auto_, Carolina_Casualty_Insurance_Co, Seneca_Specialty_Insurance_Company, ATRIUM_Certain_Underwriters_at_Lloyd_s, BindRequest, NorGuard_Insurance_Company, FT, Starr_Indemnity_STARR_, United_Fire_Casualty_Group, West_American_Insurance_Company, Great_American_Insurance_Group, Dallas_National_Insurance_Company, Ironshore_Indemnity_Inc_, Infinity_Auto_Insurance_Company, CFC_Underwriting, Founders_Ins_Co_Penn_National_, Insurance_Company_of_the_West, Leading_Insurance_Group_Ins_Co_Ltd_USB, Zenith_Insurance_Company, Merrimack_Mutual_Fire_Ins_Co_Andover_, Gemini_Insurance_Company, Safeco_Ins_Co_of_America

C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
٠٤:٢٨:٠٨.٤٢٢: AutoML: XGBoost is not available; skipping it.
٠٤:٢٨:٠٨.٤٢٦: _train param, Dropping bad and constant columns: [AHG, Mount_Vernon_Insurance_Company, Selective_Ins_Co_of_America_Selective_, Commerce_West_Insurance_Company, Fitchburg_Mutual_Insurance_Co_, Plaza_Insurance_Company_State_Auto_, Carolina_Casualty_Insurance_Co, Empire, Seneca_Specialty_Insurance_Company, BindRequest, NorGuard_Insurance_Company, FT, Selective_Way_Ins_Co_Selective_, Starr_Indemnity_STARR_, United_Fire_Casualty_Group, West_American_Insurance_Company, Great_American_Insurance_Group, Ironshore_Indemnity_Inc_, Lloyds_Canopius, CFC_Underwriting, Travelers_Indemnity_Company_of_America, Zenith_Insurance_Company, Merrimack_Mutual_Fire_Ins_Co_Andover_, Gemini_Insurance_Company, United_National_Insurance_Company, Safeco_Ins_Co_of_America_Safeco_, Chartis_Casualty_Company, RSUI_Indemnity_Company, 

...

C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
٠٤:٣٠:٤٩.٥٢٩: AutoML: XGBoost is not available; skipping it.


٠٤:٣٠:٤٩.٨٨٣: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 142.0.

███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
٠٤:٣١:٢٣.٤٢٣: AutoML: XGBoost is not available; skipping it.


٠٤:٣١:٢٣.٨٠٢: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 142.0.

███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
H2O session _sid_b678 closed.


C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
C:\Users\user\AppData\Local\Temp\ipykernel_1688\929033844.py:87: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown(prompt=False)


In [11]:
for dataset_name in data_dict_classification_only.keys():
    dataset_base = dataset_name.replace('.csv', '')
    
    csv_file1 = os.path.join(AUTOML_METRICS_PATH, f"h2o_{dataset_base}_summary.csv")
    csv_file2 = os.path.join(RUNTIME_MEMORY_TASK1_PATH, f"h2o_{dataset_base}_task1_runtime_memory_metrics.csv")

    print(f"CSV for {dataset_name.replace('.csv', '')}: h2o_{dataset_base}_summary.csv")
    df1 = pd.read_csv(csv_file1)
    display(df1)

    print(f"CSV for {dataset_name.replace('.csv', '')}: h2o_{dataset_base}_task1_runtime_memory_metrics.csv")
    df2 = pd.read_csv(csv_file2)
    display(df2)



CSV for modeldata: h2o_modeldata_summary.csv


,dataset,seed,accuracy,f1,precision,recall,runtime
0,modeldata.csv,42,0.927849,0.928492,0.931059,0.927849,31.649
1,modeldata.csv,123,0.930347,0.930832,0.932421,0.930347,31.039


CSV for modeldata: h2o_modeldata_task1_runtime_memory_metrics.csv


,dataset,seed,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,modeldata.csv,42,31.649,3.767,3010.301,0.000113,7.0
1,modeldata.csv,123,31.039,5.482,3001.473,0.000110,56.9


CSV for titanic: h2o_titanic_summary.csv


...

,dataset,seed,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,train.csv,42,31.409,0.174,2996.148,0.001015,75.9
1,train.csv,123,30.933,0.056,2996.688,0.001028,80.6


CSV for wine: h2o_wine_summary.csv


,dataset,seed,accuracy,f1,precision,recall,runtime
0,wine.csv,42,0.0,0.0,0.0,0.0,31.604
1,wine.csv,123,0.0,0.0,0.0,0.0,30.888


CSV for wine: h2o_wine_task1_runtime_memory_metrics.csv


,dataset,seed,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,wine.csv,42,31.604,3.204,2997.145,0.042961,68.6
1,wine.csv,123,30.888,0.869,2997.473,0.007089,66.7


In [12]:
# ---------------- Task 3+9 ----------------

import h2o
from h2o.automl import H2OAutoML
import torch

os.environ["JAVA_HOME"] = r"C:\Users\user\anaconda3\envs\automl_h2o"
os.environ["PATH"]  = os.path.join(os.environ["JAVA_HOME"], "Library", "bin") + ";" + os.environ["PATH"]

def evaluate_cv_folds_h2o(folds_dict):
    h2o.init(max_mem_size="2G")
    h2o.remove_all()

    for dataset_file, folds_data in folds_dict.items():
        dataset_summary = []
        runtime_memory_rows = []
        dataset_base = dataset_file.replace(".csv", "")

        for seed in RANDOM_SEEDS:
            np.random.seed(seed)
            random.seed(seed)
            try:
                torch.manual_seed(seed)
                torch.cuda.empty_cache()
            except:
                pass

            fold_num = 1
            for fold_name, fold_splits in folds_data.items():
                df_train = fold_splits['train']
                df_val   = fold_splits['val']


                h2o_train = h2o.H2OFrame(df_train)
                h2o_train[TARGET_COLS[dataset_file]] = h2o_train[TARGET_COLS[dataset_file]].asfactor()

                start_time = time.time()
                aml = H2OAutoML(max_runtime_secs=TIME_BUDGET, seed=seed, project_name=f"{dataset_base}_{seed}_{int(time.time())}")
                aml.train(y=TARGET_COLS[dataset_file], training_frame=h2o_train)
                training_runtime_sec = round(time.time() - start_time, 3)

                infer_start = time.time()
                results = evaluate_and_save_results_general(df_train, df_val, aml, dataset_file, framework='h2o')
                inference_time_per_sample = round((time.time() - infer_start) / len(df_val), 6)
                cpu_usage_percent = psutil.cpu_percent(interval=None)

                temp_model_dir = os.path.join("temp_h2o_models", f"{dataset_base}_{seed}")
                os.makedirs(temp_model_dir, exist_ok=True)
                h2o.save_model(model=aml.leader, path=temp_model_dir, force=True)

                total_bytes = 0
                for root, dirs, files in os.walk(temp_model_dir):
                    for f in files:
                        total_bytes += os.path.getsize(os.path.join(root, f))
                model_size_mb = round(total_bytes / (1024 * 1024), 3)
                shutil.rmtree(temp_model_dir)

                memory_usage_mb = get_memory_usage_mb()

                dataset_summary.append({
                    'dataset': dataset_file,
                    'seed': seed,
                    'fold': fold_num,
                    **results,
                    'runtime': training_runtime_sec
                })

                temp_model_dir = tempfile.mkdtemp()
                h2o.save_model(model=aml.leader, path=temp_model_dir, force=True)
                total_bytes = 0
                for root, dirs, files in os.walk(temp_model_dir):
                    for f in files:
                        total_bytes += os.path.getsize(os.path.join(root, f))
                model_size_mb = round(total_bytes / (1024 * 1024), 3)
                shutil.rmtree(temp_model_dir, ignore_errors=True)

                memory_usage_mb = get_memory_usage_mb()

                runtime_memory_rows.append({
                    "dataset": dataset_file,
                    "seed": seed,
                    "fold": fold_num,
                    "training_runtime_sec": training_runtime_sec,
                    "model_size_MB": model_size_mb,
                    "memory_usage_MB": memory_usage_mb,
                    "inference_time_per_sample_sec": inference_time_per_sample,
                    "cpu_usage_percent": cpu_usage_percent
                })

                fold_num += 1
                del aml, h2o_train
                gc.collect()

        output_path = os.path.join(CV_FOLDS_METRICS_PATH, f"h2o_{dataset_base}_cv_summary.csv")
        pd.DataFrame(dataset_summary).to_csv(output_path, index=False)

        task3_output_path = os.path.join(RUNTIME_MEMORY_TASK3_PATH, f"h2o_{dataset_base}_task3_runtime_memory_metrics.csv")
        pd.DataFrame(runtime_memory_rows).to_csv(task3_output_path, index=False)

        json_path = os.path.join(RUNTIME_MEMORY_TASK3_PATH, f"h2o_{dataset_base}_runtime_memory_metrics_cv_in_json.json")
        with open(json_path, 'w') as f:
            json.dump(runtime_memory_rows, f, indent=2)

    h2o.shutdown(prompt=False)

evaluate_cv_folds_h2o(folds_dict)


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Zulu25.30+17-CA (build 25.0.1+8-LTS, mixed mode, sharing)
  Starting server from C:\Users\user\anaconda3\envs\automl_h2o\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\user\AppData\Local\Temp\tmpfp3ygx9o
  JVM stdout: C:\Users\user\AppData\Local\Temp\tmpfp3ygx9o\h2o_user_started_from_python.out
  JVM stderr: C:\Users\user\AppData\Local\Temp\tmpfp3ygx9o\h2o_user_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Amman
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.8
H2O_cluster_version_age:,1 month and 26 days
H2O_cluster_name:,H2O_from_python_user_3h5qh6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.983 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
٠٤:٣٢:٣٣.٥٦٧: AutoML: XGBoost is not available; skipping it.
٠٤:٣٢:٣٣.٥٩٩: _train param, Dropping bad and constant columns: [Stillwater_Insurance_Group, Atlantic_Casualty_Insurance_Co_, XEO, C_N_A_Surety, Plaza_Insurance_Company_State_Auto_, United_States_Fire_Insurance_Company, BindRequest, Selective_Auto_Ins_Co_of_NJ_Selective_, Starr_Indemnity_STARR_, Great_American_Insurance_Group, Risk_Placement_Services_Inc_, Atrium_via_Ambris_LLLP, Underwriters_at_Lloyd_s_London_Illinois_, United_Specialty_Insurance_Co_, Infinity_Auto_Insurance_Company, Lloyds_Canopius, State_Auto_Mutual_Ins_Co, Leading_Insurance_Group_Ins_Co_Ltd_USB, Employers_Compensation_Insurance_Company, Associated_Industries_Insurance_Co_Inc, RSUI_Indemnity_Company, Everest_National_Insurance_Company, Mt_Hawley_Insurance_Company, Twin_City_Fire_Insurance_Co, Crum_Forster_Insurance, VT, Merchants_Mutual_Ins_Co_M

C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
٠٤:٣٣:٢١.٨٧٩: AutoML: XGBoost is not available; skipping it.
٠٤:٣٣:٢١.٨٨٢: _train param, Dropping bad and constant columns: [AHG, Mount_Vernon_Insurance_Company, Stillwater_Insurance_Group, BC, XEO, Great_American_Insurance_Group_of_NY, Metcom_Excess, First_Financial_Insurance_Company, Fitchburg_Mutual_Insurance_Co_, Plaza_Insurance_Company_State_Auto_, Carolina_Casualty_Insurance_Co, United_States_Fire_Insurance_Company, BindRequest, Peerless_Indemnity_Ins_Co_Peerless_, Unitrin_County_Mutual_Insurance_Company, Western_World, Sentinel_Insurance_Company_Limited, United_Fire_Casualty_Group, West_American_Insurance_Company, Rockhill_Insurance_Company, AmTrust_North_America_Inc_L_H_, Companion_Property_and_Casualty_Ins_Co, Greenwich_Insurance_Co_, Leading_Insurance_Group_Ins_Co_Ltd_USB, Associated_Industries_Insurance_Co_Inc, SureTec_Insurance_Company, United_Property_Casualty_

...

C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"



٠٤:٥٤:٥٤.٧٧٠: AutoML: XGBoost is not available; skipping it.


٠٤:٥٤:٥٥.٣٨: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 143.0.

███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |

C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"



٠٤:٥٥:٢٥.٧٩٤: AutoML: XGBoost is not available; skipping it.


٠٤:٥٥:٢٦.٧٧: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 143.0.

███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
H2O session _sid_932e closed.


C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
C:\Users\user\AppData\Local\Temp\ipykernel_1688\4103172014.py:104: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown(prompt=False)


In [13]:
for dataset_name in data_dict_classification_only.keys():
    dataset_base = dataset_name.replace('.csv', '')
    csv_file1 = os.path.join(CV_FOLDS_METRICS_PATH, f"h2o_{dataset_base}_cv_summary.csv")
    csv_file2 = os.path.join(RUNTIME_MEMORY_TASK3_PATH, f"h2o_{dataset_base}_task3_runtime_memory_metrics.csv")

    print(f"CSV for {dataset_base}: h2o_{dataset_base}_cv_summary.csv")
    df1 = pd.read_csv(csv_file1)
    display(df1)

    print(f"CSV for {dataset_base}: h2o_{dataset_base}_task3_runtime_memory_metrics.csv")
    df2 = pd.read_csv(csv_file2)
    display(df2)



CSV for modeldata: h2o_modeldata_cv_summary.csv


,dataset,seed,fold,accuracy,f1,precision,recall,runtime
0,modeldata.csv,42,1,0.927509,0.928145,0.930719,0.927509,31.849
1,modeldata.csv,42,2,0.930502,0.930688,0.931028,0.930502,31.367
2,modeldata.csv,42,3,0.926954,0.927579,0.930039,0.926954,31.352
3,modeldata.csv,42,4,0.927139,0.927553,0.928705,0.927139,31.164
4,modeldata.csv,42,5,0.932322,0.932777,0.934311,0.932322,30.621
5,modeldata.csv,123,1,0.926214,0.926959,0.930391,0.926214,30.935
6,modeldata.csv,123,2,0.928743,0.929249,0.930960,0.928743,31.329
7,modeldata.csv,123,3,0.929175,0.929296,0.929476,0.929175,30.590
8,modeldata.csv,123,4,0.927108,0.927437,0.928216,0.927108,31.280
9,modeldata.csv,123,5,0.933895,0.934237,0.935187,0.933895,30.565


CSV for modeldata: h2o_modeldata_task3_runtime_memory_metrics.csv


,dataset,seed,fold,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,modeldata.csv,42,1,31.849,1.106,3003.109,0.000118,39.2
1,modeldata.csv,42,2,31.367,1.131,3024.566,0.000109,54.4
2,modeldata.csv,42,3,31.352,1.205,3023.633,0.000108,53.8
3,modeldata.csv,42,4,31.164,1.168,3045.074,0.000108,53.6
4,modeldata.csv,42,5,30.621,5.487,3033.078,0.000109,53.5
5,modeldata.csv,123,1,30.935,1.202,3047.586,0.000108,56.7
6,modeldata.csv,123,2,31.329,1.168,3045.398,0.000115,55.3
7,modeldata.csv,123,3,30.590,4.649,3066.582,0.000114,54.1
8,modeldata.csv,123,4,31.280,5.214,3076.363,0.000121,53.1
9,modeldata.csv,123,5,30.565,4.732,3082.625,0.000124,54.3


CSV for titanic: h2o_titanic_cv_summary.csv


...

,dataset,seed,fold,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,train.csv,42,1,30.861,0.060,3065.270,0.001204,75.4
1,train.csv,42,2,30.624,10.079,3064.777,0.008355,77.0
2,train.csv,42,3,30.711,0.056,3065.516,0.001305,64.8
3,train.csv,42,4,31.308,0.178,3065.578,0.001243,59.1
4,train.csv,42,5,30.644,0.236,3065.883,0.001047,58.7
5,train.csv,123,1,31.159,9.241,3065.941,0.008094,70.2
6,train.csv,123,2,30.874,0.113,3066.223,0.001169,76.2
7,train.csv,123,3,30.413,0.180,3066.461,0.001042,59.9
8,train.csv,123,4,30.929,0.178,3066.461,0.001035,58.5
9,train.csv,123,5,30.533,0.330,3065.461,0.001292,59.2


CSV for wine: h2o_wine_cv_summary.csv


,dataset,seed,fold,accuracy,f1,precision,recall,runtime
0,wine.csv,42,1,0.000000,0.000000,0.000000,0.000000,31.040
1,wine.csv,42,2,0.000000,0.000000,0.000000,0.000000,31.739
2,wine.csv,42,3,0.000000,0.000000,0.000000,0.000000,30.843
3,wine.csv,42,4,0.085714,0.070095,0.064935,0.085714,30.818
4,wine.csv,42,5,0.000000,0.000000,0.000000,0.000000,31.224
5,wine.csv,123,1,0.000000,0.000000,0.000000,0.000000,30.909
6,wine.csv,123,2,0.027778,0.028807,0.029915,0.027778,31.158
7,wine.csv,123,3,0.000000,0.000000,0.000000,0.000000,30.698
8,wine.csv,123,4,0.028571,0.029630,0.030769,0.028571,30.875
9,wine.csv,123,5,0.000000,0.000000,0.000000,0.000000,31.193


CSV for wine: h2o_wine_task3_runtime_memory_metrics.csv


,dataset,seed,fold,training_runtime_sec,model_size_MB,memory_usage_MB,inference_time_per_sample_sec,cpu_usage_percent
0,wine.csv,42,1,31.040,0.376,3065.516,0.006808,70.3
1,wine.csv,42,2,31.739,3.355,3065.848,0.007051,72.0
2,wine.csv,42,3,30.843,0.357,3065.891,0.007004,73.7
3,wine.csv,42,4,30.818,0.986,3065.945,0.006798,70.9
4,wine.csv,42,5,31.224,2.367,3066.008,0.006860,70.5
5,wine.csv,123,1,30.909,0.383,3066.086,0.006797,68.9
6,wine.csv,123,2,31.158,11.098,3066.086,0.008777,69.1
7,wine.csv,123,3,30.698,0.267,3066.086,0.000872,68.1
8,wine.csv,123,4,30.875,0.444,3066.086,0.001065,70.5
9,wine.csv,123,5,31.193,0.764,3066.219,0.007126,68.6


In [2]:
# ---------------- Task 6 ----------------

import h2o
from h2o.automl import H2OAutoML

os.environ["JAVA_HOME"] = r"C:\Users\user\anaconda3\envs\automl_h2o"
os.environ["PATH"] = os.path.join(os.environ["JAVA_HOME"], "Library", "bin") + ";" + os.environ["PATH"]

h2o.init(max_mem_size="2G")


def safe_h2o_column_names(df):
    orig_cols = df.columns.tolist()
    new_cols = [f"f{i}" for i in range(len(orig_cols))]
    df2 = df.copy()
    df2.columns = new_cols
    mapping = dict(zip(new_cols, orig_cols))
    return df2, mapping


def evaluate_pipeline_folds_times_h2o(leakage_free_final_output):

    for dataset_name, folds in leakage_free_final_output.items():
        dataset_base = dataset_name.replace('.csv', '')

        dataset_summary = []
        time_log_rows = []

        target_col = TARGET_COLS[dataset_name]

        for seed in RANDOM_SEEDS_PIPELINE:

            np.random.seed(seed)
            random.seed(seed)

            for fold_name, split in folds.items():

                df_train = split["train"].copy()
                df_val = split["val"].copy()
                pipe_times = split["pipeline_time"]

                X_train = df_train.drop(columns=[target_col])
                y_train = df_train[target_col]

                X_val = df_val.drop(columns=[target_col])
                y_val = df_val[target_col]

                prep = pipeline_builder(dataset_name)

                X_train_proc = prep.fit_transform(X_train, y_train)
                X_train_proc = pd.DataFrame(X_train_proc)
                X_train_proc[target_col] = y_train.reset_index(drop=True)

                X_val_proc = prep.transform(X_val)
                X_val_proc = pd.DataFrame(X_val_proc)

                X_train_proc_safe, train_map = safe_h2o_column_names(X_train_proc.drop(columns=[target_col]))
                X_train_proc_safe[target_col] = y_train.values.astype(str)

                X_val_proc_safe, _ = safe_h2o_column_names(X_val_proc)

                h2o_train = h2o.H2OFrame(X_train_proc_safe)
                h2o_train[target_col] = h2o_train[target_col].asfactor()

                h2o_val = h2o.H2OFrame(X_val_proc_safe)

                start = time.time()

                aml = H2OAutoML(
                    max_runtime_secs=CV_TIME_BUDGET,
                    seed=seed,
                    project_name=f"{dataset_name}_seed{seed}_{fold_name}"
                )

                aml.train(
                    y=target_col,
                    training_frame=h2o_train
                )

                pred_df = aml.leader.predict(h2o_val).as_data_frame()
                preds = pred_df["predict"].values.flatten()

                runtime = round(time.time() - start, 3)
                fold_idx = int(fold_name.split("_")[1])

                
                best_model_raw = aml.leader.model_id
                best_model = f"h2o - {best_model_raw}"


                dataset_summary.append({
                    "dataset": dataset_name,
                    "seed": seed,
                    "fold": fold_idx,
                    "model_name": best_model,
                    "accuracy": (accuracy_score(y_val.astype(str), preds.astype(str))),
                    "f1": (f1_score(y_val.astype(str), preds.astype(str), average='weighted')),
                    "precision": (precision_score(y_val.astype(str), preds.astype(str), average='weighted')),
                    "recall": (recall_score(y_val.astype(str), preds.astype(str), average='weighted')),
                    "runtime": runtime
                })

                row = {"dataset": dataset_name, "fold": fold_idx, "model_name": best_model}
                tmp = pipe_times.copy()
                tmp["total_time"] = sum(tmp.values())
                row.update(tmp)
                time_log_rows.append(row)

        df_summary = pd.DataFrame(dataset_summary).drop_duplicates()
        df_times = pd.DataFrame(time_log_rows).drop_duplicates()

        summary_path = os.path.join(
            PIPELINE_DATAFRAMES_METRICS_PATH,
            f"h2o_{dataset_base}_pipeline_summary.csv"
        )
        df_summary.to_csv(summary_path, index=False)

        time_log_path = os.path.join(
            PIPELINE_LOG_TIMES_PATH,
            f"h2o_{dataset_base}_pipeline_time_log.csv"
        )
        df_times.to_csv(time_log_path, index=False)


evaluate_pipeline_folds_times_h2o(run_pipeline_on_folds_with_control(pipeline_data_dict))
h2o.shutdown(prompt=False)


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Zulu25.30+17-CA (build 25.0.1+8-LTS, mixed mode, sharing)
  Starting server from C:\Users\user\anaconda3\envs\automl_h2o\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\user\AppData\Local\Temp\tmpfw72xr8m
  JVM stdout: C:\Users\user\AppData\Local\Temp\tmpfw72xr8m\h2o_user_started_from_python.out
  JVM stderr: C:\Users\user\AppData\Local\Temp\tmpfw72xr8m\h2o_user_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,Asia/Amman
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.8
H2O_cluster_version_age:,1 month and 24 days
H2O_cluster_name:,H2O_from_python_user_tyb0r4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.984 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
١٨:٣٩:٢٨.٦٢٨: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
١٨:٤٠:٠١.٦٢٠: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


...

C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"



١٨:٤١:٤٤.٥٢٨: AutoML: XGBoost is not available; skipping it.
١٨:٤١:٤٤.٦٧٦: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 75.0.

███████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |

C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"



١٨:٤١:٥٤.٩٩٨: AutoML: XGBoost is not available; skipping it.
١٨:٤١:٥٥.١٣٩: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 75.0.

███████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
H2O session _sid_93df closed.


C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
C:\Users\user\AppData\Local\Temp\ipykernel_18812\2846575204.py:126: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown(prompt=False)


In [2]:
# ---------------- Task 6 ----------------

import h2o
from h2o.automl import H2OAutoML
import torch

os.environ["JAVA_HOME"] = r"C:\Users\user\anaconda3\envs\automl_h2o"
os.environ["PATH"] = os.path.join(os.environ["JAVA_HOME"], "Library", "bin") + ";" + os.environ["PATH"]

h2o.init(max_mem_size="4G", nthreads=-1)
h2o.remove_all()

def safe_h2o_column_names(df):
    orig_cols = df.columns.tolist()
    new_cols = [f"f{i}" for i in range(len(orig_cols))]
    df2 = df.copy()
    df2.columns = new_cols
    mapping = dict(zip(new_cols, orig_cols))
    return df2, mapping

def evaluate_pipeline_folds_times_h2o(leakage_free_final_output):
    for dataset_name, folds in leakage_free_final_output.items():
        dataset_base = dataset_name.replace('.csv', '')

        runtime_memory_rows = []
        dataset_summary = []

        target_col = TARGET_COLS[dataset_name]

        for seed in RANDOM_SEEDS_PIPELINE:
            np.random.seed(seed)
            random.seed(seed)
            try:
                torch.manual_seed(seed)
                torch.cuda.empty_cache()
            except:
                pass

            for fold_name, split in folds.items():
                df_train = split["train"]
                df_val = split["val"]
                pipe_times = split["pipeline_time"]

                X_train = df_train.drop(columns=[target_col])
                y_train = df_train[target_col]

                X_val = df_val.drop(columns=[target_col])
                y_val = df_val[target_col]

                prep = pipeline_builder(dataset_name)

                X_train_proc = pd.DataFrame(prep.fit_transform(X_train, y_train))
                X_train_proc[target_col] = y_train.reset_index(drop=True)
                X_val_proc = pd.DataFrame(prep.transform(X_val))

                X_train_safe, _ = safe_h2o_column_names(X_train_proc.drop(columns=[target_col]))
                X_train_safe[target_col] = y_train.values.astype(str)
                X_val_safe, _ = safe_h2o_column_names(X_val_proc)

                h2o_train = h2o.H2OFrame(X_train_safe)
                h2o_train[target_col] = h2o_train[target_col].asfactor()
                h2o_val = h2o.H2OFrame(X_val_safe)

                start = time.time()
                aml = H2OAutoML(
                    max_runtime_secs=CV_TIME_BUDGET,
                    seed=seed,
                    project_name=f"{dataset_name}_seed{seed}_{fold_name}"
                )
                aml.train(y=target_col, training_frame=h2o_train)
                runtime = round(time.time() - start, 3)

                pred_df = aml.leader.predict(h2o_val).as_data_frame()
                preds = pred_df["predict"].values.flatten()
                fold_idx = int(fold_name.split("_")[1])
                best_model = f"h2o - {aml.leader.model_id}"

                dataset_summary.append({
                    "dataset": dataset_name,
                    "seed": seed,
                    "fold": fold_idx,
                    "model_name": best_model,
                    "accuracy": accuracy_score(y_val.astype(str), preds.astype(str)),
                    "f1": f1_score(y_val.astype(str), preds.astype(str), average='weighted'),
                    "precision": precision_score(y_val.astype(str), preds.astype(str), average='weighted'),
                    "recall": recall_score(y_val.astype(str), preds.astype(str), average='weighted'),
                    "runtime": runtime
                })

                row = {"dataset": dataset_name, "fold": fold_idx, "model_name": best_model}
                tmp = pipe_times.copy()
                tmp["total_time"] = sum(tmp.values())
                row.update(tmp)
                runtime_memory_rows.append(row)

                h2o.remove(h2o_train)
                h2o.remove(h2o_val)
                del aml
                gc.collect()

        df_summary = pd.DataFrame(dataset_summary).drop_duplicates()
        df_times = pd.DataFrame(runtime_memory_rows).drop_duplicates()

        df_summary.to_csv(os.path.join(
            PIPELINE_DATAFRAMES_METRICS_PATH,
            f"h2o_{dataset_base}_pipeline_summary.csv"
        ), index=False)

        df_times.to_csv(os.path.join(
            PIPELINE_LOG_TIMES_PATH,
            f"h2o_{dataset_base}_pipeline_time_log.csv"
        ), index=False)

    h2o.shutdown(prompt=False)

evaluate_pipeline_folds_times_h2o(run_pipeline_on_folds_with_control(pipeline_data_dict))

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Zulu25.30+17-CA (build 25.0.1+8-LTS, mixed mode, sharing)
  Starting server from C:\Users\user\anaconda3\envs\automl_h2o\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\user\AppData\Local\Temp\tmp3nab5ucy
  JVM stdout: C:\Users\user\AppData\Local\Temp\tmp3nab5ucy\h2o_user_started_from_python.out
  JVM stderr: C:\Users\user\AppData\Local\Temp\tmp3nab5ucy\h2o_user_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Amman
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.8
H2O_cluster_version_age:,1 month and 26 days
H2O_cluster_name:,H2O_from_python_user_ey09i4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.983 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
١١:٤٣:٢٣.٥٤٤: AutoML: XGBoost is not available; skipping it.
١١:٤٣:٢٣.٥٧٢: _train param, Dropping bad and constant columns: [f0]

█
١١:٤٣:٢٤.٨٢٩: _train param, Dropping bad and constant columns: [f0]

██████████████████████████████
١١:٤٣:٢٨.٩٦٤: _train param, Dropping unused columns: [f0]
١١:٤٣:٢٩.٦٤٩: _train param, Dropping bad and constant columns: [f0]

█████████
١١:٤٣:٣٠.٤٥٥: _train param, Dropping bad and constant columns: [f0]

█████████
١١:٤٣:٣١.١٩٢: _train param, Dropping bad and constant columns: [f0]
١١:٤٣:٣١.٩٠١: _train param, Dropping bad and constant columns: [f0]

████████
١١:٤٣:٣٢.٦١١: _train param, Dropping unused columns: [f0]

██████| (done) 100%

١١:٤٣:٣٢.٩٧٨: _train param, Dropping unused columns: [f0]
١١:٤٣:٣٣.٤٧٦: _train param, Dropping bad and constant colum

C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
١١:٤٣:٤٣.١٠٩: AutoML: XGBoost is not available; skipping it.
١١:٤٣:٤٣.١١١: _train param, Dropping bad and constant columns: [f0]


١١:٤٣:٤٣.٨٢١: _train param, Dropping bad and constant columns: [f0]

████████████
١١:٤٣:٤٦.٧٤٥: _train param, Dropping unused columns: [f0]

█████
١١:٤٣:٤٧.١٠٣: _train param, Dropping bad and constant columns: [f0]

██████
١١:٤٣:٤٨.٢٤٣: _train param, Dropping bad and constant columns: [f0]

████████
١١:٤٣:٤٩.٣٥٨: _train param, Dropping bad and constant columns: [f0]

█████████
١١:٤٣:٥٠.٤٦٠: _train param, Dropping bad and constant columns: [f0]

█████████
١١:٤٣:٥١.٥٦٧: _train param, Dropping unused columns: [f0]

████████
١١:٤٣:٥١.٩١٤: _train param, Dropping unused columns: [f0]
١١:٤٣:٥٢.٣٩٨: _train param, Dropping bad and constant columns: [f0]

████
١

...

C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"



١١:٤٦:٢٤.٩٦٩: AutoML: XGBoost is not available; skipping it.


١١:٤٦:٢٥.١٩٤: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 89.0.

███████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |

C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"



١١:٤٦:٣٥.٨٨٩: AutoML: XGBoost is not available; skipping it.
١١:٤٦:٣٦.١٠٠: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 89.0.

███████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
H2O session _sid_b9e1 closed.


C:\Users\user\anaconda3\envs\automl_h2o\lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
C:\Users\user\AppData\Local\Temp\ipykernel_18940\3073994901.py:114: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown(prompt=False)


In [3]:
for dataset_name in data_dict_classification_only.keys():
    dataset_base = dataset_name.replace('.csv', '')
    
    csv_file_1 = os.path.join(PIPELINE_DATAFRAMES_METRICS_PATH, f"h2o_{dataset_base}_pipeline_summary.csv")
    csv_file_2 = os.path.join(PIPELINE_LOG_TIMES_PATH, f"h2o_{dataset_base}_pipeline_time_log.csv")
    
    df1 = pd.read_csv(csv_file_1)
    df2 = pd.read_csv(csv_file_2)
    
    print(f"CSV for {dataset_base}: h2o_{dataset_base}_pipeline_summary.csv")
    display(df1) 
    print(f"CSV for {dataset_base}: h2o_{dataset_base}_pipeline_time_log.csv")
    display(df2) 



CSV for modeldata: h2o_modeldata_pipeline_summary.csv


,dataset,seed,fold,model_name,accuracy,f1,precision,recall,runtime
0,modeldata.csv,42,1,h2o - StackedEnsemble_AllModels_1_AutoML_1_b`b...,0.920563,0.920583,0.920605,0.920563,11.019
1,modeldata.csv,42,2,h2o - GBM_4_AutoML_2_b`beab`e_aadcdc,0.922759,0.923462,0.926262,0.922759,11.051
2,modeldata.csv,123,1,h2o - StackedEnsemble_AllModels_1_AutoML_3_b`b...,0.920230,0.919977,0.919909,0.920230,10.956
3,modeldata.csv,123,2,h2o - StackedEnsemble_BestOfFamily_3_AutoML_4_...,0.924375,0.925032,0.927578,0.924375,10.822


CSV for modeldata: h2o_modeldata_pipeline_time_log.csv


,dataset,fold,model_name,preprocessing,nan_guard_before_vif,vif,binner,nan_guard_before_poly,selector,total_time
0,modeldata.csv,1,h2o - StackedEnsemble_AllModels_1_AutoML_1_b`b...,0.454725,0.143556,0.012527,0.049021,0.046427,0.023232,0.729487
1,modeldata.csv,2,h2o - GBM_4_AutoML_2_b`beab`e_aadcdc,0.410254,0.138704,0.012081,0.048611,0.045486,0.022144,0.677281
2,modeldata.csv,1,h2o - StackedEnsemble_AllModels_1_AutoML_3_b`b...,0.454725,0.143556,0.012527,0.049021,0.046427,0.023232,0.729487
3,modeldata.csv,2,h2o - StackedEnsemble_BestOfFamily_3_AutoML_4_...,0.410254,0.138704,0.012081,0.048611,0.045486,0.022144,0.677281


CSV for titanic: h2o_titanic_pipeline_summary.csv


...

,dataset,fold,model_name,preprocessing,nan_guard_before_vif,vif,binner,nan_guard_before_poly,poly,selector,total_time
0,train.csv,1,h2o - GBM_grid_1_AutoML_9_b`beab`e_aadeah_mode...,0.001003,0.0,0.000000,0.001002,0.0,0.0,0.0,0.002005
1,train.csv,2,h2o - GLM_1_AutoML_10_b`beab`e_aadebi,0.001004,0.0,0.001002,0.000000,0.0,0.0,0.0,0.002006
2,train.csv,1,h2o - DeepLearning_grid_1_AutoML_11_b`beab`e_a...,0.001003,0.0,0.000000,0.001002,0.0,0.0,0.0,0.002005
3,train.csv,2,h2o - GBM_grid_1_AutoML_12_b`beab`e_aadeea_mod...,0.001004,0.0,0.001002,0.000000,0.0,0.0,0.0,0.002006


CSV for wine: h2o_wine_pipeline_summary.csv


,dataset,seed,fold,model_name,accuracy,f1,precision,recall,runtime
0,wine.csv,42,1,h2o - GBM_5_AutoML_13_b`beab`e_aadf`c,0.955056,0.954885,0.955118,0.955056,10.786
1,wine.csv,42,2,h2o - DeepLearning_grid_2_AutoML_14_b`beab`e_a...,0.853933,0.854115,0.866148,0.853933,10.881
2,wine.csv,123,1,h2o - GBM_grid_1_AutoML_15_b`beab`e_aadfbd_mod...,0.932584,0.932299,0.934580,0.932584,10.795
3,wine.csv,123,2,h2o - DeepLearning_grid_1_AutoML_16_b`beab`e_a...,0.696629,0.663687,0.739606,0.696629,10.809


CSV for wine: h2o_wine_pipeline_time_log.csv


,dataset,fold,model_name,preprocessing,nan_guard_before_vif,vif,binner,nan_guard_before_poly,poly,selector,total_time
0,wine.csv,1,h2o - GBM_5_AutoML_13_b`beab`e_aadf`c,0.002267,0.0,0.000000,0.001002,0.0,0.0,0.0,0.003269
1,wine.csv,2,h2o - DeepLearning_grid_2_AutoML_14_b`beab`e_a...,0.002004,0.0,0.001002,0.001002,0.0,0.0,0.0,0.004008
2,wine.csv,1,h2o - GBM_grid_1_AutoML_15_b`beab`e_aadfbd_mod...,0.002267,0.0,0.000000,0.001002,0.0,0.0,0.0,0.003269
3,wine.csv,2,h2o - DeepLearning_grid_1_AutoML_16_b`beab`e_a...,0.002004,0.0,0.001002,0.001002,0.0,0.0,0.0,0.004008
